<a href="https://colab.research.google.com/github/Cchancee/summative_MLOP/blob/main/summative_mlop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# CIFAR already has shape (32, 32, 3), so no reshaping needed

# Class labels
class_names = [
    "Airplane", "Automobile", "Bird", "Cat", "Deer",
    "Dog", "Frog", "Horse", "Ship", "Truck"
]

# Build CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train
history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=15
)

Epoch 1/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 72s 51ms/step - accuracy: 0.5093 - loss: 1.3636 - val_accuracy: 0.6260 - val_loss: 1.1001
Epoch 2/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 68s 49ms/step - accuracy: 0.5558 - loss: 1.2423 - val_accuracy: 0.6358 - val_loss: 1.0481
Epoch 3/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 84s 50ms/step - accuracy: 0.5823 - loss: 1.1687 - val_accuracy: 0.6586 - val_loss: 0.9950
Epoch 4/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - accuracy: 0.6108 - loss: 1.1212 - val_accuracy: 0.6690 - val_loss: 0.9568
Epoch 5/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 83s 48ms/step - accuracy: 0.6198 - loss: 1.0758 - val_accuracy: 0.6850 - val_loss: 0.9251
Epoch 6/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 80s 47ms/step - accuracy: 0.6298 - loss: 1.0518 - val_accuracy: 0.6748 - val_loss: 0.9370
Epoch 7/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - accuracy: 0.6334 - loss: 1.0278 - val_accuracy: 0.7024 - val_loss: 0.8810
Epoch 8/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - accuracy: 0.6446 -

In [ ]:

# Test
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.6990 - loss: 0.8606
Test accuracy: 0.7026


In [ ]:
from sklearn.metrics import classification_report

# Predict classes
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_flat = y_test.flatten()  # from shape (10000, 1) to (10000,)

# Report
print(classification_report(y_test_flat, y_pred_classes, target_names=class_names))


313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step
              precision    recall  f1-score   support

    Airplane       0.73      0.75      0.74      1000
  Automobile       0.88      0.77      0.82      1000
        Bird       0.62      0.54      0.58      1000
         Cat       0.50      0.51      0.51      1000
        Deer       0.67      0.61      0.64      1000
         Dog       0.65      0.55      0.60      1000
        Frog       0.70      0.83      0.76      1000
       Horse       0.76      0.75      0.76      1000
        Ship       0.74      0.88      0.81      1000
       Truck       0.74      0.83      0.79      1000

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
model.save("models/cifar10_cnn.h5")